<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.1: Introduction to FIRRTL

**Prev: [Generators: Types](3.6_types.ipynb)**<br>
**Next: [FIRRTL AST Traversal](4.2_firrtl_ast_traversal.ipynb)**

## Motivation
You've learned some Scala and written some Chisel, and for 90% of users, that should be enough to become a Chisel aficionado.

However, some use cases are better expressed as a programmatic transformation of a Chisel design, rather than as a generator.

For example, suppose we want to count the number of registers in a design. This would be difficult to do as a generator, so instead, we can write a FIRRTL pass to do it for us.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


path: String = "/home/peter/AIU/AIU_CS800_Chisel/300_UCB_BC/004.1_FIRRTL_Intro/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import firrtl._

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import firrtl._

## What is FIRRTL?
As you've probably become aware, when you execute a Chisel design, it elaborates (executes the surrounding Scala code) to construct an instance of your generator, with all Scala parameters resolved.

Instead of directly emitting Verilog, Chisel emits an intermediate representation called FIRRTL, which represents the elaborated (parameter-resolved) RTL instance. It can be serialized (converted to a String for writing to a file), and this serialized syntax is human readable. Internally, however, it is not represented as a long string. Instead, it is a datastructure organized as a tree of nodes, called an abstract-syntax-tree (AST).

Let's take a look! We will take a simple Chisel design, elaborate it, and inspect what FIRRTL it generates!

First, we define a Chisel module, which delays its input signal by two cycles.

In [3]:
class DelayBy2(width: Int) extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  val r0 = RegNext(io.in)
  val r1 = RegNext(r0)
  io.out := r1
}

defined class DelayBy2

Next, let's elaborate it, serialize, and print out the FIRRTL it generates.

In [4]:
println(chisel3.Driver.emit(() => new DelayBy2(4)))

Elaborating design...
Done elaborating.
circuit DelayBy2 :
  module DelayBy2 :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    reg r0 : UInt, clock with :
      reset => (UInt<1>("h0"), r0) @[cmd2.sc 6:19]
    r0 <= io.in @[cmd2.sc 6:19]
    reg r1 : UInt, clock with :
      reset => (UInt<1>("h0"), r1) @[cmd2.sc 7:19]
    r1 <= r0 @[cmd2.sc 7:19]
    io.out <= r1 @[cmd2.sc 8:10]



As you can see, the serialized FIRRTL looks very similar to what our Chisel design would look like, with all generator parameters resolved.

## The FIRRTL AST

As mentioned earlier, the FIRRTL representation can be serialized as a String, but internally, it is a datastructure called an AST (abstract syntax tree). This data structure is a tree of nodes, where one node can contain children nodes. There are no cycles in this datastructure.

Let's take a look at what the internal datastructure looks like:

In [5]:
val firrtlSerialization = chisel3.Driver.emit(() => new DelayBy2(4))
val firrtlAST = firrtl.Parser.parse(firrtlSerialization.split("\n").toIterator, Parser.GenInfo("file.fir"))

println(firrtlAST)

Elaborating design...
Done elaborating.
Circuit( @[file.fir 1:0],ArrayBuffer(Module( @[file.fir 2:2],DelayBy2,ArrayBuffer(Port( @[file.fir 3:4],clock,Input,ClockType), Port( @[file.fir 4:4],reset,Input,UIntType(IntWidth(1))), Port( @[file.fir 5:4],io,Output,BundleType(ArrayBuffer(Field(in,Flip,UIntType(IntWidth(4))), Field(out,Default,UIntType(IntWidth(4))))))),Block(ArrayBuffer(DefRegister( @[file.fir 7:4],r0,UIntType(UnknownWidth),Reference(clock,UnknownType,UnknownKind,UnknownFlow),UIntLiteral(0,IntWidth(1)),Reference(r0,UnknownType,UnknownKind,UnknownFlow)), Connect( @[file.fir 9:4],Reference(r0,UnknownType,UnknownKind,UnknownFlow),SubField(Reference(io,UnknownType,UnknownKind,UnknownFlow),in,UnknownType,UnknownFlow)), DefRegister( @[file.fir 10:4],r1,UIntType(UnknownWidth),Reference(clock,UnknownType,UnknownKind,UnknownFlow),UIntLiteral(0,IntWidth(1)),Reference(r1,UnknownType,UnknownKind,UnknownFlow)), Connect( @[file.fir 12:4],Reference(r1,UnknownType,UnknownKind,UnknownFlow),Ref

firrtlSerialization: String = """circuit DelayBy2 :
  module DelayBy2 :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    reg r0 : UInt, clock with :
      reset => (UInt<1>("h0"), r0) @[cmd2.sc 6:19]
    r0 <= io.in @[cmd2.sc 6:19]
    reg r1 : UInt, clock with :
      reset => (UInt<1>("h0"), r1) @[cmd2.sc 7:19]
    r1 <= r0 @[cmd2.sc 7:19]
    io.out <= r1 @[cmd2.sc 8:10]
"""
firrtlAST: ir.Circuit = Circuit(
  FileInfo("file.fir 1:0"),
  ArrayBuffer(
    Module(
      FileInfo("file.fir 2:2"),
      "DelayBy2",
      ArrayBuffer(
        Port(FileInfo("file.fir 3:4"), "clock", Input, ClockType),
        Port(FileInfo("file.fir 4:4"), "reset", Input, UIntType(IntWidth(1))),
        Port(
          FileInfo("file.fir 5:4"),
          "io",
          Output,
          BundleType(
            ArrayBuffer(
              Field("in", Flip, UIntType(IntWidth(4))),
              Field("out", Default, UIntType(IntWidth(4)))
            

Obviously, the serialization of a datastructure isn't as pretty, but you can see some of the classes and such that internally represent the RTL design. Let's try to pretty that up a bit to make it understandable.

In [6]:
println(stringifyAST(firrtlAST))

Circuit(
| @[file.fir1:0],
| ArrayBuffer(
| | Module(
| | | @[file.fir2:2],
| | | DelayBy2,
| | | ArrayBuffer(
| | | | Port(
| | | | | @[file.fir3:4],
| | | | | clock,
| | | | | Input,
| | | | | ClockType
| | | | ),
| | | | Port(
| | | | | @[file.fir4:4],
| | | | | reset,
| | | | | Input,
| | | | | UIntType(
| | | | | | IntWidth(
| | | | | | | 1
| | | | | | )
| | | | | )
| | | | ),
| | | | Port(
| | | | | @[file.fir5:4],
| | | | | io,
| | | | | Output,
| | | | | BundleType(
| | | | | | ArrayBuffer(
| | | | | | | Field(
| | | | | | | | in,
| | | | | | | | Flip,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | ),
| | | | | | | Field(
| | | | | | | | out,
| | | | | | | | Default,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | )
| | | | | | )
| | | | | )
| | | | )
| | | ),
| | | Block(
| | | | ArrayBuffer(
| | | | | DefRegister(
| |

This is the internal datastructure that holds the FIRRTL AST. It is a tree structure whose root node is **Circuit**, which has 3 children: **@[file.fir@2.0]**, **ArrayBuffer**, and **cmd5WrapperHelperDelayBy2**. The following is the definition of `Circuit`'s actual Scala class that was serialized:<a name="circuit"></a><img src="images/circuit.png" alt="Circuit case class" />



As you can see, it has three children nodes: `info: Info`, `Modules: Seq[DefModule]`, and `main: String`. It extends `FirrtlNode`, of which all FIRRTL AST nodes must do. Ignore the `def mapXXXX` functions for now.

Many FIRRTL nodes contain an `info: Info` field, which the parser can either insert file information like line number and column number, or insert a `NoInfo` token. In this example, **@[file.fir@2.0]** would refer to the FIRRTL file, line 2, column 0.

The following section will outline all of these FIRRTL nodes in detail.

# FIRRTL Node Descriptions

This section describes common FirrtlNodes found in [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala).

For more detail on components not mentioned here, please refer to [The FIRRTL Specification](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf).


## Circuit
Circuit is the root node of any Firrtl datastructure. There is only ever one Circuit, and that Circuit contains a list of module definitions and the name of the top-level module.

#### FirrtlNode Declaration
```scala 
Circuit(info: Info, modules: Seq[DefModule], main: String)
```

#### Concrete Syntax
```
circuit Adder:
  ... //List of modules
```
#### In-memory Representation
```scala
Circuit(NoInfo, Seq(...), "Adder")
```

## Module

Modules are the unit of modularity within Firrtl and are never directly nested (declaring an instance of a module has its own concrete syntax and AST representation). Each Module has a name, and a list of ports, and a body containing its implementation.

#### FirrtlNode declaration
```scala
Module(info: Info, name: String, ports: Seq[Port], body: Stmt) extends DefModule
```

#### Concrete Syntax
```
module Adder:
  ... // list of ports
  ... // statements
```
#### In-memory representation
```scala
Module(NoInfo, "Adder", Seq(...), )
```

## Port
A port defines part of a Module's io, and has a name, direction (input or output), and type.

#### FirrtlNode Declaration
```scala
class Port(info: Info, name: String, direction: Direction, tpe: Type)
```
#### Concrete Syntax
```
input x: UInt
```

#### In-memory representation
```scala
Port(NoInfo, "x", INPUT, UIntType(UnknownWidth))
```

## Statement
A statement is used to describe the components within a module and how they interact. Below are some commonly used statements:

### Block of Statements
A group of statements. Commonly used as the body field in a Module declaration.

### Wire Declaration
A wire declaration, containing a name and type. It can be both a source (connected *from*) and a sink (connected *to").
#### FirrtlNode declaration
```scala
DefWire(info: Info, name: String, tpe: Type)
```
#### Concrete syntax
```
wire w: UInt
```
#### In-memory Representation
```scala
DefWire(NoInfo, "w", UIntType(UnknownWidth))
```

### Register Declaration
A register declaration, containing a name, type, clock signal, reset signal, and reset value.
#### FirrtlNode declaration
```scala
DefRegister(info: Info, name: String, tpe: Type, clock: Expression, reset: Expression, init: Expression)
```

### Connection
Represents a directioned connection from a source to a sink. Note that it abides by last-connect-semantics, as described in Chisel.

#### FirrtlNode declaration
```scala
Connect(info: Info, loc: Expression, expr: Expression)
```

### Other Statements
Other statement types like `DefMemory`, `DefNode`, `IsInvalid`, `Conditionally`, and others are omitted here; please refer to [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/freechipsproject/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala) for more detail.

## Expression
Expressions represent references to declared components or logical and arithmetic operations. Below are some commonly used expressions:

### Reference
A reference to a declared component, such as a wire, register, or port. It has a name and type field. Note that it does not contain a pointer to the actual declaration, but instead just contains the name as a String.

#### FirrtlNode declaration
```scala
Reference(name: String, tpe: Type)
```

### DoPrim
An anonymous primitive operation, such as `Add`, `Sub`, or `And`, `Or`, or subword-selection (`Bits`). The type of operation is indicated by the `op: PrimOp` field. Note that the number of required arguments and constants are determined by the `op`.

#### FirrtlNode declaration
```scala
DoPrim(op: PrimOp, args: Seq[Expression], consts: Seq[BigInt], tpe: Type)
```

### Other Expressions
Other expressions including `SubField`, `SubIndex`, `SubAccess`, `Mux`, `ValidIf` etc. are described in more detail in [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala) and [The FIRRTL Specification](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf).

# Back to our example

Let's take another look at the FIRRTL AST from our example. Hopefully, the structure of the design makes more sense!

In [7]:
println(stringifyAST(firrtlAST))

Circuit(
| @[file.fir1:0],
| ArrayBuffer(
| | Module(
| | | @[file.fir2:2],
| | | DelayBy2,
| | | ArrayBuffer(
| | | | Port(
| | | | | @[file.fir3:4],
| | | | | clock,
| | | | | Input,
| | | | | ClockType
| | | | ),
| | | | Port(
| | | | | @[file.fir4:4],
| | | | | reset,
| | | | | Input,
| | | | | UIntType(
| | | | | | IntWidth(
| | | | | | | 1
| | | | | | )
| | | | | )
| | | | ),
| | | | Port(
| | | | | @[file.fir5:4],
| | | | | io,
| | | | | Output,
| | | | | BundleType(
| | | | | | ArrayBuffer(
| | | | | | | Field(
| | | | | | | | in,
| | | | | | | | Flip,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | ),
| | | | | | | Field(
| | | | | | | | out,
| | | | | | | | Default,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | )
| | | | | | )
| | | | | )
| | | | )
| | | ),
| | | Block(
| | | | ArrayBuffer(
| | | | | DefRegister(
| |

That's it for this section! In the next section, we will look at how a FIRRTL transformation walks this AST and modifies it.